In [ ]:
from vector_store.initialize import create_vector_store, initialize_embedding_model
from vector_store.insert import upload_content_to_store
from langchain_core.tools import create_retriever_tool
from apps.model_app import aya_gradio_chat
import gradio as gr

Keys loaded successfully

Device being used in the embedding model is cpu



# Upload content Steps

In [7]:
embedding_model = initialize_embedding_model()

store = create_vector_store(
    persist_path="./store/aya_healthcare",
    collection_name="aya_healthcare_rag",
    embedding_model=embedding_model
)


Device being used in the embedding model is cpu



In [5]:
upload_content_to_store(store=store, documents_path="outputs/updated_headings.md")

Successfully Loaded single file: outputs\updated_headings.md


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Successfully added files from outputs/updated_headings.md to vector store


# Retriever Tool Steps

In [9]:
store_retriever = store.as_retriever(search_type="mmr", search_kwargs={"k": 3})

retriever_tool = create_retriever_tool(
    retriever=store_retriever,
    name="aya_healthcare_database",
    description="Used for searching through stored information about Aya Healthcare scraped from Aya websites to aid in user queries",
    response_format="content_and_artifact"
)


# APP

In [ ]:
# Customize the chatbot display
custom_chatbot = gr.Chatbot(
    height=600,                    # Height in pixels
    show_label=False,              # Hide the "Chatbot" label above
    avatar_images=(None, "🤖"),    # (user_avatar, bot_avatar) - can use emoji or image paths
    editable='user',       # Keep bubbles from stretching full width
    buttons=['copy'],         # Add copy button to messages
    watermark='Mine bitch',
    layout="bubble",                # "panel" or "bubble" style
    placeholder="Start chatting!"  # Message shown when chat is empty
)

# Customize the input textbox
custom_textbox = gr.Textbox(
    placeholder="Type your message here...",
    container=False,               # Remove the container border
    scale=7,                       # Takes up more horizontal space relative to buttons
    lines=1,                       # Number of visible lines (expands as user types)
    max_lines=10,                  # Maximum lines before scrolling
    autofocus=True,                 # Auto-focus on page load
    submit_btn='Submit',
    stop_btn='Stop'
)

gr.ChatInterface(
    fn=aya_gradio_chat,
    chatbot=custom_chatbot,
    textbox=custom_textbox,
    title='Aya Chatbot'
).launch(share=True, theme=gr.themes.Ocean())